<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/Dataset_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Datasets

In [1]:
import pickle
import os
from sklearn.model_selection import train_test_split
import pandas as pd

## Utils

In [2]:
def sample_subset(df, n):
    """
    Given a dataframe with label, text and description columns,
    samples up to n samples for each label to a subset of the dataset.
    """
    return df.groupby('label').apply(lambda x: x.sample(n=min(n, len(x)))).reset_index(drop=True)


## ATIS

Pickle files can be downloaded from https://www.kaggle.com/code/siddhadev/atis-dataset-from-ms-cntk

In [3]:
!wget https://github.com/elephanti/NLPProject2024/raw/main/atis.test.pkl -P inputs/ATIS
!wget https://github.com/elephanti/NLPProject2024/raw/main/atis.train.pkl -P inputs/ATIS

--2024-07-18 09:15:35--  https://github.com/elephanti/NLPProject2024/raw/main/atis.test.pkl
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/elephanti/NLPProject2024/main/atis.test.pkl [following]
--2024-07-18 09:15:35--  https://raw.githubusercontent.com/elephanti/NLPProject2024/main/atis.test.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277306 (271K) [application/octet-stream]
Saving to: ‘inputs/ATIS/atis.test.pkl’

atis.test.pkl       100%[===================>] 270.81K  --.-KB/s    in 0.1s    

2024-07-18 09:15:36 (2.01 MB/s) - ‘inputs/ATIS/atis.test.pkl’ saved [277306/277306]

--2024

### Load dataset

In [4]:
def load_atis(fname, exclude_joint_categories=False):
    with open(fname, 'rb') as stream:
        ds,dicts = pickle.load(stream)
    print('Done  loading: ', fname)
    print('      samples: {:4d}'.format(len(ds['query'])))
    print('   vocab_size: {:4d}'.format(len(dicts['token_ids'])))
    print('   slot count: {:4d}'.format(len(dicts['slot_ids'])))
    print(' intent count: {:4d}'.format(len(dicts['intent_ids'])))

    t2i, s2i, in2i = map(dicts.get, ['token_ids', 'slot_ids','intent_ids'])
    i2t, i2s, i2in = map(lambda d: {d[k]:k for k in d.keys()}, [t2i,s2i,in2i])
    query, slots, intent =  map(ds.get, ['query', 'slot_labels', 'intent_labels'])

    samples = []

    for i in range(len(query)):
        label = i2in[intent[i][0]]

        if exclude_joint_categories and "+" in label:
            continue
        text = list(map(i2t.get, query[i]))

        # Filter BOS and EOS
        if text[0] == 'BOS' and text[-1] == 'EOS':
            text = text[1:-1]

        samples.append({
            "label": label,
            "text": ' '.join(text),
            "description": None
        })

    # save samples csv
    df = pd.DataFrame(samples)
    return df

In [5]:
train_df = load_atis(os.path.join('inputs/ATIS','atis.train.pkl'), True)
test_df = load_atis(os.path.join('inputs/ATIS','atis.test.pkl'), True)
atis_df = pd.concat([train_df, test_df])

os.makedirs('datasets/ATIS', exist_ok=True)
atis_df.to_csv('datasets/ATIS/atis.full.csv', index=False)

# split to train 80%, validation 10% test 10%
atis_train_df, atis_test_df = train_test_split(atis_df, test_size=0.2)
atis_valid_df, atis_test_df = train_test_split(atis_test_df, test_size=0.5)

atis_train_df.to_csv('datasets/ATIS/atis.train.csv', index=False)
atis_valid_df.to_csv('datasets/ATIS/atis.valid.csv', index=False)
atis_test_df.to_csv('datasets/ATIS/atis.test.csv', index=False)

Done  loading:  inputs/ATIS/atis.train.pkl
      samples: 4978
   vocab_size:  943
   slot count:  129
 intent count:   26
Done  loading:  inputs/ATIS/atis.test.pkl
      samples:  893
   vocab_size:  943
   slot count:  129
 intent count:   26


In [6]:
atis_df.head()

,label,text,description
0,flight,i want to fly from boston at 838 am and arrive...,None
1,flight,what flights are available from pittsburgh to ...,None
2,flight_time,what is the arrival time in san francisco for ...,None
3,airfare,cheapest airfare from tacoma to orlando,None
4,airfare,round trip fares from pittsburgh to philadelph...,None


In [7]:
atis_df['label'].unique()

array(['flight', 'flight_time', 'airfare', 'aircraft', 'ground_service',
       'airport', 'airline', 'distance', 'abbreviation', 'ground_fare',
       'quantity', 'city', 'flight_no', 'capacity', 'meal', 'restriction',
       'cheapest', 'day_name'], dtype=object)

In [8]:
# Print for each intent how many samples there are
atis_df.groupby('label').count()

,text,description
label,,
abbreviation,180,0
aircraft,90,0
airfare,471,0
airline,195,0
airport,38,0
capacity,37,0
cheapest,1,0
city,25,0
day_name,2,0


### Sample subsets



In [9]:
for ver in range(1, 6):
  os.makedirs(f'datasets/ATIS/sampled_subsets/ver{ver}', exist_ok=True)

  atis_5_subset = sample_subset(atis_train_df, 5)
  atis_5_subset.to_csv(f'datasets/ATIS/sampled_subsets/ver{ver}/atis_5_subset_ver.csv', index=False)
  atis_10_subset = sample_subset(atis_train_df, 10)
  atis_10_subset.to_csv(f'datasets/ATIS/sampled_subsets/ver{ver}/atis_10_subset.csv', index=False)
  atis_20_subset = sample_subset(atis_train_df, 20)
  atis_20_subset.to_csv(f'datasets/ATIS/sampled_subsets/ver{ver}/atis_20_subset.csv', index=False)
  atis_50_subset = sample_subset(atis_train_df, 50)
  atis_50_subset.to_csv(f'datasets/ATIS/sampled_subsets/ver{ver}/atis_50_subset.csv', index=False)
  atis_100_subset = sample_subset(atis_train_df, 100)
  atis_100_subset.to_csv(f'datasets/ATIS/sampled_subsets/ver{ver}/atis_100_subset.csv', index=False)

## TREC

### Load dataset

In [10]:
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/train_1000.label -P inputs/TREC
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/train_2000.label -P inputs/TREC
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/train_3000.label -P inputs/TREC
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/train_4000.label -P inputs/TREC
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label -P inputs/TREC
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label -P inputs/TREC

--2024-07-18 09:15:39--  https://cogcomp.seas.upenn.edu/Data/QA/QC/train_1000.label
Resolving cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)... 158.130.57.77
Connecting to cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)|158.130.57.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60774 (59K)
Saving to: ‘inputs/TREC/train_1000.label’

train_1000.label    100%[===================>]  59.35K   157KB/s    in 0.4s    

2024-07-18 09:15:40 (157 KB/s) - ‘inputs/TREC/train_1000.label’ saved [60774/60774]

--2024-07-18 09:15:40--  https://cogcomp.seas.upenn.edu/Data/QA/QC/train_2000.label
Resolving cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)... 158.130.57.77
Connecting to cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)|158.130.57.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122681 (120K)
Saving to: ‘inputs/TREC/train_2000.label’

train_2000.label    100%[===================>] 119.81K   218KB/s    in 0.5s    

2024-07-18 09:15:41 (21

In [11]:
import pandas as pd
import os

# Process into a df
def load_trec(filename):
    samples = []
    with open(filename, 'r', encoding='windows-1252') as f:
        for line in f:
            label, text = line.split(' ', 1)
            samples.append({
                "label": label,
                "text": text.strip(),
                "description": None
            })
    return pd.DataFrame(samples)

train_1000_df = load_trec('inputs/TREC/train_1000.label')
train_2000_df = load_trec('inputs/TREC/train_2000.label')
train_3000_df = load_trec('inputs/TREC/train_3000.label')
train_4000_df = load_trec('inputs/TREC/train_4000.label')
train_5500_df = load_trec('inputs/TREC/train_5500.label')
test_df = load_trec('inputs/TREC/TREC_10.label')

trec_df = pd.concat([train_1000_df, train_2000_df, train_3000_df, train_4000_df, train_5500_df, test_df])

os.makedirs('datasets/TREC', exist_ok=True)
trec_df.to_csv('datasets/TREC/trec.full.csv', index=False)

# split to train 80%, validation 10% test 10%
trec_train_df, trec_test_df = train_test_split(trec_df, test_size=0.2)
trec_valid_df, trec_test_df = train_test_split(trec_test_df, test_size=0.5)

trec_train_df.to_csv('datasets/TREC/trec.train.csv', index=False)
trec_valid_df.to_csv('datasets/TREC/trec.valid.csv', index=False)
trec_test_df.to_csv('datasets/TREC/trec.test.csv', index=False)

trec_df.head()


,label,text,description
0,DESC:manner,How did serfdom develop in and then leave Russ...,None
1,ENTY:cremat,What films featured the character Popeye Doyle ?,None
2,DESC:manner,How can I find a list of celebrities ' real na...,None
3,ENTY:animal,What fowl grabs the spotlight after the Chines...,None
4,ABBR:exp,What is the full form of .com ?,None


In [12]:
trec_df['label'].unique()

array(['DESC:manner', 'ENTY:cremat', 'ENTY:animal', 'ABBR:exp', 'HUM:ind',
       'HUM:gr', 'HUM:title', 'DESC:def', 'NUM:date', 'DESC:reason',
       'ENTY:event', 'LOC:state', 'DESC:desc', 'NUM:count', 'ENTY:other',
       'ENTY:letter', 'LOC:other', 'ENTY:religion', 'ENTY:food',
       'LOC:country', 'ENTY:color', 'ENTY:termeq', 'LOC:city',
       'ENTY:body', 'ENTY:dismed', 'LOC:mount', 'NUM:money',
       'ENTY:product', 'NUM:period', 'ENTY:substance', 'ENTY:sport',
       'ENTY:plant', 'ENTY:techmeth', 'NUM:volsize', 'HUM:desc',
       'ENTY:instru', 'ABBR:abb', 'NUM:other', 'NUM:speed', 'ENTY:word',
       'ENTY:lang', 'NUM:perc', 'NUM:code', 'NUM:dist', 'NUM:temp',
       'ENTY:symbol', 'NUM:ord', 'ENTY:veh', 'NUM:weight',
       'ENTY:currency'], dtype=object)

In [13]:
trec_df.groupby('label').count()

,text,description
label,,
ABBR:abb,47,0
ABBR:exp,203,0
DESC:def,1344,0
DESC:desc,781,0
DESC:manner,768,0
DESC:reason,549,0
ENTY:animal,381,0
ENTY:body,56,0
ENTY:color,129,0


### Sample subsets

In [14]:
for ver in range(1, 6):
  os.makedirs(f'datasets/TREC/sampled_subsets/ver{ver}', exist_ok=True)

  trec_5_subset = sample_subset(trec_train_df, 5)
  trec_5_subset.to_csv(f'datasets/TREC/sampled_subsets/ver{ver}/trec_5_subset.csv', index=False)
  trec_10_subset = sample_subset(trec_train_df, 10)
  trec_10_subset.to_csv(f'datasets/TREC/sampled_subsets/ver{ver}/trec_10_subset.csv', index=False)
  trec_20_subset = sample_subset(trec_train_df, 20)
  trec_20_subset.to_csv(f'datasets/TREC/sampled_subsets/ver{ver}/trec_20_subset.csv', index=False)
  trec_50_subset = sample_subset(trec_train_df, 50)
  trec_50_subset.to_csv(f'datasets/TREC/sampled_subsets/ver{ver}/trec_50_subset.csv', index=False)
  trec_100_subset = sample_subset(trec_train_df, 100)
  trec_100_subset.to_csv(f'datasets/TREC/sampled_subsets/ver{ver}/trec_100_subset.csv', index=False)

## Push to Github

In [15]:
!git config --global user.email 'email'
!git config --global user.name 'username'

!git clone https://<USERNAME>:<TOKEN>@github.com/elephanti/NLPProject2024.git
%cd NLPProject2024
!cp -r /content/datasets /content/NLPProject2024/datasets
!cp -r /content/inputs /content/NLPProject2024/inputs

!git add datasets
!git add inputs
!git commit -m "Add generated datasets"
!git push

Cloning into 'NLPProject2024'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 152.86 KiB | 756.00 KiB/s, done.
/content/NLPProject2024
fatal: /content/datasets: '/content/datasets' is outside repository at '/content/NLPProject2024'
fatal: /content/inputs: '/content/inputs' is outside repository at '/content/NLPProject2024'
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	datasets/
	inputs/

nothing added to commit but untracked files present (use "git add" to track)
Everything up-to-date
